# Compression

Compression reduces or increases dynamic range by reducing or increasing an input signal once a threshold is breached.

Components of compression:

- Level detection: measure amplitude
- Gain: boost signal as compression will reduce volume
- Smoothing (attack/release): prevent abrupt changes
- Apply gain: multiply input by computed gain


In [ ]:
import numpy as np
import warnings
from IPython.display import Audio, display
from scipy.io import wavfile
import matplotlib.pyplot as plt

og_len = 5000 # 5 seconds
channels = 2  # Stereo audio
sr = 44100 # stream rate
audio_path = "../../resources/drums1.wav"

warnings.simplefilter("ignore", wavfile.WavFileWarning)
sr_loaded, y = wavfile.read(audio_path)  # y has shape (samples, channels) if stereo

# Handle stereo by taking mean
if y.ndim == 2:
    y = y.mean(axis=1)

# Normalize to -1 to 1 range
waveform = y.T.astype(np.float32) / np.max(np.abs(y))

# Sample window for envelope follower
window_size = int(0.01 * sr)  # 10ms sampling window

# Rectify inputs
# We need absolute values when we calculate the gain reduction amount in decibels
rectified = np.abs(waveform)


# Peak Detection Compression

In [ ]:
# Linear scale threshold
# Values above the threshold will be compressed
 # linear scale threshold of 0.5 ~ -6dB
threshold = 0.05
ratio = 4.0

# This convolves the rectified input signal with averaging window to get the envelope
envelope = np.convolve(rectified, np.ones(window_size)/window_size, mode='same')

# Gather signal and calculate the target gain
# If the envelope is above the threshold, reduce the gain according to the ratio
# If the envelope is below the threshold, just return the envelope (no gain change)
gain = np.where(
  envelope > threshold,
  threshold + (envelope - threshold) / ratio,
  envelope
)

# Calculate the gain reduction factor for each sample
gain = gain / (envelope + 1e-8)

# Apply the gain reduction to the original signal
# After this step, the signal has been compressed
compressed_signal = waveform * gain

print("Dry:")
display(Audio(waveform, rate=sr))

print("Harsh Compression:")
display(Audio(compressed_signal, rate=sr))

plt.figure(figsize=(12, 6))
plt.title("Peak Compression")
plt.plot(waveform[:5000], label="Original waveform")
plt.plot(compressed_signal[:5000], label="Compressed waveform", alpha=0.8)
plt.plot(gain[:5000], label="Gain factor", alpha=0.8)
plt.legend()
plt.show()


# RMS Detection

In [ ]:
threshold = 0.08
ratio = 4.0

# RMS envelope follower (smoother, more transparent)
rms_envelope = np.sqrt(
    np.convolve(waveform**2, np.ones(window_size)/window_size, mode='same')
)

gain = np.where(
  envelope > threshold,
  threshold + (rms_envelope - threshold) / ratio,
  envelope
)

# Calculate the gain reduction factor for each sample
gain = gain / (rms_envelope + 1e-8)

# Apply the gain reduction to the original signal
# After this step, the signal has been compressed
compressed_signal = waveform * gain

print("RMS Compression:")
display(Audio(compressed_signal, rate=sr))

plt.figure(figsize=(12, 6))
plt.title("RMS Compression")
plt.plot(waveform[:5000], label="Original waveform")
plt.plot(compressed_signal[:5000], label="Compressed waveform", alpha=0.8)
plt.plot(gain[:5000], label="Gain factor", alpha=0.8)
plt.legend()
plt.show()


# RMS Detection + Attack and Release

In [ ]:
threshold = 0.08
ratio = 4.0

# Attack: how quickly gain reduction is applied (fast = 1-10ms)
attack_time = 0.005  # 5ms
attack_samples = int(attack_time * sr)
attack_coeff = np.exp(-1.0 / attack_samples)

# Release: how quickly gain reduction recovers (slow = 50-500ms)
release_time = 0.1  # 100ms
release_samples = int(release_time * sr)
release_coeff = np.exp(-1.0 / release_samples)

rms_envelope = np.sqrt(
    np.convolve(waveform**2, np.ones(window_size)/window_size, mode='same')
)

# Apply attack/release envelope follower
gain_smooth = np.zeros_like(rms_envelope)
gain_smooth[0] = rms_envelope[0]

for i in range(1, len(envelope)):
    if rms_envelope[i] > gain_smooth[i-1]:
        # Attack (signal getting louder)
        coeff = attack_coeff
    else:
        # Release (signal getting quieter)
        coeff = release_coeff
    
    gain_smooth[i] = coeff * gain_smooth[i-1] + (1 - coeff) * rms_envelope[i]

# Now use gain_smooth instead of envelope for compression
gain = np.where(
    gain_smooth > threshold,
    threshold + (gain_smooth - threshold) / ratio,
    gain_smooth
)
gain = gain / (gain_smooth + 1e-8)

compressed_signal = waveform * gain

print("Attack/Release + RMS Compression")
display(Audio(compressed_signal, rate=sr))

plt.figure(figsize=(12, 6))
plt.title("Attack and Release + RMS Peak Compression")
plt.plot(waveform[:5000], label="Original waveform")
plt.plot(compressed_signal[:5000], label="Compressed waveform", alpha=0.8)
plt.plot(gain[:5000], label="Gain factor", alpha=0.8)
plt.legend()
plt.show()